In [ ]:
## model_id = "openai-community/gpt2-large"
## dataset_dpo = load_dataset("yuasosnin/imdb-dpo", split="train[90%:]")

In [ ]:
# install all dependencies
%%capture

!pip install openai
!pip install -q -U peft transformers datasets bitsandbytes trl accelerate
!pip install --upgrade transformers, datasets==2.16.1, accelerate==0.26.1, evaluate==0.4.1, bitsandbytes==0.42.0, trl==0.7.11, peft==0.8.2


In [ ]:
# Library
%%capture

from huggingface_hub import hf_hub_download

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import TrainingArguments
from peft import LoraConfig, AutoPeftModelForCausalLM
from datasets import load_dataset, Dataset
from trl import SFTTrainer, DPOTrainer

from huggingface_hub import notebook_login
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

import numpy as np
from preference_datasets import get_dataset
import datasets
from utils import TemporarilySeededRandom
from copy import deepcopy
# Ignore warings

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# log in to the Hugging Face hub (required for private datasets/models)
notebook_login()

In [ ]:
# Load the 7b llama-2 model
model_id = "openai-community/gpt2-large"

# Set quantization config (to save memory)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

# Load model, quantized
base_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map={"": 0})
base_model.config.use_cache = False

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, device_map={"": 0})

# Set it to a new token to correctly attend to EOS tokens.
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training


In [ ]:
# load dataset
dataset_dpo = load_dataset("jondurbin/truthy-dpo-v0.1", split="train[900:]")

print(dataset_dpo.shape)

df_dpo = dataset_dpo.to_pandas()
df_dpo.head()

# keep rows with 'system' column = 'You are an unbiased, uncensored, helpful assistant.'
# df_dpo = df_dpo[df_dpo["system"] == "You are an unbiased, uncensored, helpful assistant."]
df_dpo.head()

# keep only columns 'prompt', 'chosen', 'rejected'
df_dpo = df_dpo[["prompt", "chosen", "rejected"]]

# change every text in promt from str to user: str. asistent:
df_dpo["prompt"] = df_dpo["prompt"].apply(lambda x: "### USER: " + x + "\n### ASSISTANT: ")
filtered_dataset = Dataset.from_pandas(df_dpo)

# random  seed 42
filtered_dataset = filtered_dataset.shuffle(seed=42)

print(df_dpo.shape)
df_dpo.head()


In [ ]:

# Define LoRA ("low-rank attention") config
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)


In [ ]:
## Load my tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained('gpt2-large')
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

## load my model
huggingface_filepath = hf_hub_download(repo_id="your/model/id", filename="policy.pt")

model_test = transformers.AutoModelForCausalLM.from_pretrained('gpt2-large')
model_test.load_state_dict(torch.load(huggingface_filepath, map_location=torch.device('cuda'))['state'])


In [ ]:
model_test = model_test.eval()

n_epochs = 1
n_examples = None
rank = 0
seed = 0
assert n_epochs is not None or n_examples is not None, "Must specify either n_epochs or n_examples"
silent = False
split = 'test'

if silent:
    datasets.logging.disable_progress_bar()
    datasets.logging.set_verbosity_error()

#collate_fn = get_collate_fn(tokenizer)
reward_counter = 0
counter = 0
epoch_idx = 0
example_idx = 0
gen_counter = 0
total_arrays = {}
batch_size = 4
max_length = 512
max_prompt_length=256
sft_mode = True
done = False
data_storage = []

while True:
    if n_epochs is not None and epoch_idx >= n_epochs:
        if not silent:
            print(f'Finished generating {n_epochs} epochs on {split} split')
        break

    batch = []
    for prompt in df_dpo['prompt']:
        if done:
            break
        else:

            model_inputs = tokenizer(prompt, return_tensors='pt')
            greedy_output = model_test.generate(**model_inputs, max_new_tokens=40)

            policy_output = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
            policy_output = policy_output.removeprefix(prompt)
            print(policy_output)
            data_storage.append({'prompt': prompt, 'res':policy_output, 'response':prompt + policy_output})
            print('counter')
            print(counter)
            counter += 1

            if counter == 100:
              done = True
    epoch_idx += 1



In [ ]:
#creating json1 file for measure_reward script
import json
from google.colab import files

with open("your_json_file.jsonl","w") as outfile:
  json.dump(data_storage,outfile,ensure_ascii=False)


files.download('your_json_file.jsonl')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
## Load my tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained('gpt2-large')
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

test_model_id = "your/model/id"

tokenizer = AutoTokenizer.from_pretrained(test_model_id)
model_test = AutoModelForCausalLM.from_pretrained(test_model_id)

In [ ]:
model_test_2 = model_test.eval()

n_epochs = 1
n_examples = None
rank = 0
seed = 0
assert n_epochs is not None or n_examples is not None, "Must specify either n_epochs or n_examples"
silent = False
split = 'test'

if silent:
    datasets.logging.disable_progress_bar()
    datasets.logging.set_verbosity_error()

#collate_fn = get_collate_fn(tokenizer)
reward_counter = 0
counter = 0
epoch_idx = 0
example_idx = 0
gen_counter = 0
total_arrays = {}
batch_size = 4
max_length = 512
max_prompt_length=256
sft_mode = True
done = False
data_storage = []

while True:
    if n_epochs is not None and epoch_idx >= n_epochs:
        if not silent:
            print(f'Finished generating {n_epochs} epochs on {split} split')
        break

    batch = []
    for prompt in df_dpo['prompt']:
        if done:
            break
        else:

            model_inputs = tokenizer(prompt, return_tensors='pt')
            greedy_output = model_test_2.generate(**model_inputs, max_new_tokens=40)

            policy_output = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
            policy_output = policy_output.removeprefix(prompt)
            if counter % 10 == 0:
              print(policy_output)
            data_storage.append({'prompt': prompt, 'res':policy_output, 'response':prompt + policy_output})
            print(counter)
            counter += 1

            if counter == 100:
              done = True
    epoch_idx += 1



In [ ]:
#creating json1 file for measure_reward script
import json
from google.colab import files

with open("your_json_file.jsonl","w") as outfile:
  json.dump(data_storage,outfile,ensure_ascii=False)


files.download('your_json_file.jsonl')

In [ ]:
!python gpt4-eval.py --run_name_red your_json_file --run_name_blue your_json_file